In [1]:
from flask import Flask, render_template, request
import nltk
import json
import random
from nltk.stem import WordNetLemmatizer
import pickle
import numpy as np
from keras.models import load_model

In [2]:
model = load_model('chatbot_model.h5')
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [ ]:
app = Flask(__name__)

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/get")
def get_bot_response(show_details=True):
    userText = request.args.get('msg')
    
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(userText)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]

    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    p = np.array(bag)
    
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    
    ints = return_list
    
    tag = ints[0]['intent']
    list_of_intents = intents['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return str(result.lower())

if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [02/Apr/2024 12:32:03] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Apr/2024 12:32:04] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [02/Apr/2024 12:32:49] "GET /person.png HTTP/1.1" 404 -


found in bag: hi
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step


INFO:werkzeug:127.0.0.1 - - [02/Apr/2024 12:32:51] "GET /get?msg=hi HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Apr/2024 12:32:51] "GET /bot.png HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [02/Apr/2024 12:33:00] "GET /person.png HTTP/1.1" 404 -


found in bag: ppsu
found in bag: location
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


...

INFO:werkzeug:127.0.0.1 - - [02/Apr/2024 12:34:09] "GET /get?msg=hi HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Apr/2024 12:34:09] "GET /bot.png HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [02/Apr/2024 12:34:16] "GET /person.png HTTP/1.1" 404 -


found in bag: how
found in bag: are
found in bag: you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step


INFO:werkzeug:127.0.0.1 - - [02/Apr/2024 12:34:16] "GET /get?msg=how%20are%20you%3F HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Apr/2024 12:34:16] "GET /bot.png HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [02/Apr/2024 12:34:49] "GET /person.png HTTP/1.1" 404 -


found in bag: what
found in bag: is
found in bag: your
found in bag: name
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


INFO:werkzeug:127.0.0.1 - - [02/Apr/2024 12:34:49] "GET /get?msg=what%20is%20your%20name%3F HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Apr/2024 12:34:49] "GET /bot.png HTTP/1.1" 404 -
